In [1]:
# from google import google
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
from sqlalchemy import MetaData
from configparser import ConfigParser
import os
import spacy
nlp = spacy.load('en')
import re
import phonenumbers
import contextlib

from imwithdata.es_etl.issues_actions import (
    issues,
    actions,
    state_regex,
    city_regex,
    web_url_regex,
    date_include_regex,
    date_exclude_regex,
    leg_name_regex,
    leg_twitter_regex
)

In [3]:
# states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas','District of Columbia','California', 'Colorado', 
#           'Connecticut', 'Delaware', 'Florida', 'Georgia', 
#           'Hawaii', 'Idaho','Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 
#           'Massachusetts','Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 
#           'New Hampshire','New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 
#           'Oregon','Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
#           'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming','ALABAMA', 'ALASKA', 'ARIZONA', 
#           'ARKANSAS','DISTRICT OF COLUMBIA','CALIFORNIA', 'COLORADO', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'GEORGIA', 
#           'HAWAII', 'IDAHO','ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND', 
#           'MASSACHUSETTS','MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 'NEBRASKA', 'NEVADA', 
#           'NEW HAMPSHIRE','NEW JERSEY', 'NEW MEXICO', 'NEW YORK', 'NORTH CAROLINA', 'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 
#           'OREGON','PENNSYLVANIA', 'RHODE ISLAND', 'SOUTH CAROLINA', 'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'UTAH',
#           'VERMONT', 'VIRGINIA', 'WASHINGTON', 'WEST VIRGINIA', 'WISCONSIN', 'WYOMING','NY','NYC']

# state_abbrevs = ['IA', 'KS', 'UT', 'MA','MI','MO',
#           'VA', 'NC', 'NE', 'SD', 'AL', 'ID', 'FM', 'DE', 'AK', 'Conn', 'NM', 'MS', 'GA',
#           'CO', 'NJ', 'FL', 'MN', 'NV', 'AZ', 'WI', 'ND', 'OK', 'KY', 'RI', 'NH', 'MO', 'ME', 'VT',
#           'NY', 'CA', 'HI', 'IL', 'TN', 'Mass', 'OH', 'MD', 'MI', 'WY', 'WA', 'SC','PA',
#           'IN', 'LA', 'DC', 'AR', 'WV', 'TX','Minn','Ark','Ind','D\.C\.',
#           'Mizz','Nev','Okl','Penn','S\. Carolina','N\. Carolina','S Carolina','N Carolina','Tenn','Tex']

# state_regex = re.compile(r'(\\b' + '|\\b'.join(states) + '|' + '|'.join([state + '\\b' for state in states])
#                                                     + '|' + '|'.join(['\\b' + state + '\\b' for state in state_abbrevs])+ r')')
# print (state_regex)

In [57]:
config_file = os.path.join(os.pardir,'config','config.ini')

def get_ini_vals(ini_file, section):
    config = ConfigParser()
    config.read(ini_file)
    return config[section]

mysql_creds = get_ini_vals(config_file, 'mysql')

engine = create_engine("""mysql+pymysql://{user}:{password}@{host}:{port}/{db}""".format(user=mysql_creds['user'],
                                                                                 password=mysql_creds['password'],
                                                                                 host=mysql_creds['host'],
                                                                                 port=mysql_creds['port'],
                                                                                 db=mysql_creds['database']
                                                                                )
                      )
conn = engine.connect()


In [5]:
final_data = pd.read_csv('/Users/brosskatz/PycharmProjects/rzst/w210_imwithdata/imwithdata/data/static_data/final_data_example.csv')
# final_data.columns
final_data.drop(final_data.columns[0], axis=1,inplace=True)
# final_data.columns
actions = final_data.sort('total_score',ascending=[0])
actions.head()


issue_list = actions['issue'].tolist()
action_list = actions['action'].tolist()
id_list = actions['id'].tolist()

es_score_list = actions['es_score'].tolist()
total_score_list = actions['total_score'].tolist()
tweet_list = [tweet.capitalize() if tweet.isupper() else tweet for tweet in actions['tweet'].tolist() ]
tweet_timestamp_list = actions['tweet_timestamp'].tolist()
query_timestamp_list = actions['query_timestamp'].tolist()
user_list = actions['tweet_user'].tolist()


/Users/brosskatz/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:5: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [5]:
print (len(final_data))

227


In [6]:
time_regex = re.compile(r'\d{1,2}(?:(?:am|pm)|(?::\d{1,2})(?:am|pm)?)', re.IGNORECASE)

dates = []
start_times = []
end_times = []
cities = []
states = []
legislators = []
legislator_handles = []
phone_numbers = []
titles = []
all_urls = []

for i, tweet in enumerate(tweet_list):
    date = ''
    start_time = ''
    end_time = ''
    city = ''
    state = ''
    legislator = ''
    legislator_handle = ''
    phone_number = ''
    url = ''
    title = ''
    
    doc = nlp(tweet)
    all_dates = [doc.text for doc in doc.ents if doc.label_ == 'DATE']
    date_matches = re.findall(date_include_regex, ' '.join(all_dates))
    ### EXCLUDE SOME DIRTY DATES FROM TWITTER THAT SPACY MISTAKENLY INCLUDES    
    if date_matches:
        date = all_dates[0]
    dates.append(date)
        
    times = re.findall(time_regex,tweet)
    if times:
        start_time = times[0]
    if len(times) > 1:
        end_time = times[1]
    start_times.append(start_time)
    end_times.append(end_time)
    
    tweet_cities = re.findall(city_regex,tweet)
    if tweet_cities:
        tweet_cities = list(set([city.title() for city in tweet_cities]))
        if len(tweet_cities) == 1:
            city = tweet_cities[0]
        else:
            city = '; '.join(tweet_cities)
    cities.append(city)
    
    tweet_states = re.findall(state_regex,tweet)
    if tweet_states:
        tweet_states = list(set(tweet_states))
        if len(tweet_states) == 1:
            state = tweet_states[0]
        else:
            state = '; '.join(tweet_states)
    states.append(state)
    
    tweet_legislators = re.findall(leg_name_regex,tweet)
    if tweet_legislators:
        if len(tweet_legislators) == 1:
            legislator = tweet_legislators[0]
        else:
            legislator = '; '.join(tweet_legislators)
    legislators.append(legislator)
    
    tweet_leg_handles = re.findall(leg_twitter_regex,tweet)
    if tweet_leg_handles:
        if len(tweet_leg_handles) == 1:
            legislator_handle = tweet_leg_handles[0]
        else:
            legislator_handle = '; '.join(tweet_leg_handles)
    legislator_handles.append(legislator_handle)
    
    if phonenumbers.PhoneNumberMatcher(tweet, "US"):
        for i,match in enumerate(phonenumbers.PhoneNumberMatcher(tweet, "US")):
            if i == 0:  
                phone_number = phonenumbers.format_number(match.number,
                                                            phonenumbers.PhoneNumberFormat.NATIONAL)
    phone_numbers.append(phone_number)
    
    urls = ''
    tweet_urls = re.findall(web_url_regex,tweet)
    if tweet_urls:
        tweet_urls = list(set(tweet_urls))
        if len(tweet_urls) == 1:
            urls = tweet_urls[0]
        else:
            urls = '; '.join(tweet_urls)
    all_urls.append(urls)
    
    
    title = tweet
    if tweet_urls:
        for urly in tweet_urls:
            title = title.replace(urly,'')
    titles.append(title)


In [10]:
city_states = []

for i, city in enumerate(cities):
    city_state = ''
    state = states[i]
    if city != '':
        if state == '':
            city_state = city
        elif state != '':
            city_state = '; '.join(list(set([cit + ', ' + state.split('; ')[0] for cit in city.split('; ')])))
        else:
            city_state = city
    else:
        if state == city:
            city_state = city
        elif state != '':
            city_state = state
            
    if city_state == 'Washington, Washington':
        city_state = 'Washington, DC'
    if city_state == 'Mobile':
        city_state = ''
    city_states.append(city_state)

In [11]:
for i, city_state in enumerate(city_states[:1000]):
    print (city_state)
    print (tweet_list[i])
    print ()

California
California D.A.'s should reveal their financial support from Planned Parenthood. https://t.co/TWJ9uEF29r


Join @NRDC &amp; sign petition to defend our environment from Trump and his fossil fuel allies in Congress. https://t.co/PwbVSJzSyW


Protect Reproductive Health And Rights  #sign #petition to Stand With Planned Parenthood &gt; https://t.co/50eRJVFMgX

California
California girls and boys are the best... boycott the rest https://t.co/CHc45cjTaR

Washington, DC
Washington Post worried about DC environment.... Let them donate or otherwise volunteer to remedy the situation instead of pointing fingers


Let's go people sign petition for this Union to put Trump out.Because the rules of the election was broken. Which was voting equality rights

Sacramento; Los Angeles
Los Angeles Times Protests erupt at Sacramento town hall meeting as ICE director answers… https://t.co/RruNs5a2av


Worth a listen to at 26:30 a Holocaust survivor setting straight the ICE director &amp; the loc

In [12]:
#### ACTIONS SEND TO SQL

prepped_to_sql = pd.DataFrame(
                        {'id': id_list,
                         'title': [title.encode('utf-8') for title in titles],
                         'description': [tweet.encode('utf-8') for tweet in tweet_list],
                         'action': [action.encode('utf-8') for action in action_list],
                         'issue': [issue.encode('utf-8') for issue in issue_list],
                         'total_score':total_score_list,
                         'relevance_score':es_score_list,
                         'legislators': legislators,
                         'legislator_twitter': legislator_handles,
                         'city':cities,
                         'state':states,
                         'city_state':city_states,
                         'phone_number': phone_numbers,
                         'urls': [url.encode('utf-8') for url in all_urls],
                         'date_of_action': dates,
                         'announced_date': tweet_timestamp_list,
                         'query_date':query_timestamp_list
                        })

prepped_to_sql.to_sql('rzst_action',conn,if_exists='replace',index=False)

In [22]:
import sunlight
from sunlight import congress
import pandas as pd

sunlight.config.API_KEY = 'thisisakey'
legislators = congress.all_legislators_in_office()

In [15]:
all_keys = set().union(*(d.keys() for d in legislators))
print(all_keys)

{'votesmart_id', 'fax', 'state_rank', 'ocd_id', 'website', 'facebook_id', 'icpsr_id', 'govtrack_id', 'first_name', 'bioguide_id', 'district', 'in_office', 'last_name', 'senate_class', 'term_start', 'fec_ids', 'youtube_id', 'state', 'chamber', 'birthday', 'title', 'crp_id', 'gender', 'middle_name', 'leadership_role', 'lis_id', 'other_names', 'term_end', 'party', 'office', 'thomas_id', 'phone', 'twitter_id', 'nickname', 'state_name', 'oc_email', 'contact_form', 'name_suffix'}


In [33]:
legislator_df = pd.DataFrame(legislators)
final_legislator_df = legislator_df[['title','first_name','last_name','state','state_name','party','website',
                                     'phone','oc_email','contact_form']]
final_legislator_df.sort_values(['state_name'], ascending=[ True],inplace=True)

/Users/brosskatz/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [35]:
final_legislator_df.to_sql('rzst_legislators',conn,if_exists='replace',index=False)

In [54]:
townhalls = pd.read_csv('/Users/brosskatz/PycharmProjects/rzst/w210_imwithdata/imwithdata/data/static_data/Townhalls.csv')

In [58]:
townhalls.columns = map(str.lower, townhalls.columns)
townhalls.columns

Index(['event_title', 'event_description', 'event_location', 'event_date',
       'event_time', 'event_url', 'event_related_state', 'event_legislator',
       'event_meeting_type'],
      dtype='object')

In [56]:
townhalls['event_title'] = townhalls['event_legislator'] + ' ' + townhalls['event_meeting_type']

In [57]:
townhalls = townhalls[['event_title',
                       'event_description',
                       'event_location',
                       'event_date',
                       'event_time',
                       'event_url',
                       'event_related_state',
                       'event_legislator',
                       'event_meeting_type'
                      ]]

In [59]:
townhalls.to_sql('rzst_events',conn,if_exists='replace',index=False)

In [4]:
# cities = pd.read_csv(os.path.join(os.pardir, 'data/static_data', 'Top5000Population.csv'))
# cities['full_city'] = cities['city'] + ', ' + cities['state'] 
# print cities[:10]
# city_list = list(cities['full_city'])
# print city_list[:10]
# # print city_list.index('Jersey City, NJ')/

# searches = [city + ' indivisible' for city in city_list[35:]]
# print searches[:10]

In [9]:

# num_page = 1
# # nresults = 0
# # today = datetime.datetime.today()
# # weeks_ago = today - datetime.timedelta(days=14)

# indivisible_list = []

# for search in searches:
#     try:
#         search_results = google.search(search, num_page)
#         for result in search_results:
#             new_record = {
#             'name' : result.name,  # The title of the link
#             'link' : result.link, # The external link
#             'google_link': result.google_link, # The google link
#             'description': result.description, # The description of the link
#             'thumbnail': result.thumb, # The link to a thumbnail of the website (NOT implemented yet)
#             'search': search
#           }
#             indivisible_list.append(new_record)
#     except:
#         break


Error accessing: http://www.google.com/search?q=Lancaster+%2C+CA+indivisible&start=0&num=10&nl=en
Error accessing: http://www.google.com/search?q=Fort+Collins+%2C+CO+indivisible&start=0&num=10&nl=en
Error accessing: http://www.google.com/search?q=Warren+%2C+MI+indivisible&start=0&num=10&nl=en
Error accessing: http://www.google.com/search?q=Hartford+%2C+CT+indivisible&start=0&num=10&nl=en
Error accessing: http://www.google.com/search?q=McKinney+%2C+TX+indivisible&start=0&num=10&nl=en
Error accessing: http://www.google.com/search?q=Visalia+%2C+CA+indivisible&start=0&num=10&nl=en
Error accessing: http://www.google.com/search?q=Killeen+%2C+TX+indivisible&start=0&num=10&nl=en
Error accessing: http://www.google.com/search?q=Evansville+%2C+IN+indivisible&start=0&num=10&nl=en
Error accessing: http://www.google.com/search?q=Vallejo+%2C+CA+indivisible&start=0&num=10&nl=en
Error accessing: http://www.google.com/search?q=Ann+Arbor+%2C+MI+indivisible&start=0&num=10&nl=en
Error accessing: http://www

In [5]:
# indivisible_df = pd.DataFrame(indivisible_list)
# mask = ((indivisible_df['link'].str.contains("facebook.com/")) | (indivisible_df['link'].str.contains("meetup.com/")) | (indivisible_df['link'].str.contains("//indivisible")) | (indivisible_df['link'].str.contains("//www.indivisible"))) & ~(indivisible_df['link'].str.contains("indivisibleguide.com"))
# indivisible_orgs = indivisible_df[mask]
# print (indivisible_orgs[:30])
# indivisible_orgs.to_csv('~/w210_imwithdata/imwithdata/data/static_data/indivisible_orgs_search_results2.csv',encoding='utf-8')


In [6]:
indivisible_df = pd.read_csv('/Users/brosskatz/PycharmProjects/rzst/w210_imwithdata/imwithdata/data/static_data/indivisible_orgs_search_results.csv',encoding='utf-8')
indivisible_df2 = pd.read_csv('/Users/brosskatz/PycharmProjects/rzst/w210_imwithdata/imwithdata/data/static_data/indivisible_orgs_search_results2.csv',encoding='utf-8')


In [7]:
indivisible_df2.columns

Index(['Unnamed: 0', 'description', 'google_link', 'link', 'name', 'search',
       'thumbnail'],
      dtype='object')

In [8]:
indivisible_df.drop(indivisible_df.columns[0], axis=1,inplace=True)
indivisible_df2.drop(indivisible_df2.columns[0], axis=1,inplace=True)

In [9]:
full_indivisible_df = indivisible_df2.append(indivisible_df, ignore_index=True)

In [10]:
len(full_indivisible_df)

1033

In [11]:
full_indivisible_df.drop_duplicates(subset =['link'],inplace=True)

In [34]:
mask = ~(full_indivisible_df['link'].str.contains("event")) & ~(full_indivisible_df['link'].str.contains("topics")) & ~(full_indivisible_df['link'].str.contains("cities")) & ~(full_indivisible_df['link'].str.contains("movement"))
full_indivisible_df = full_indivisible_df[mask]


In [36]:
len(full_indivisible_df)

408

In [48]:

urls = full_indivisible_df['link'].tolist()
names = full_indivisible_df['name'].tolist()
descriptions = [ names[i] + ' ' + desc for i,desc in enumerate(full_indivisible_df['description'].tolist())]
searches = full_indivisible_df['search'].tolist()
google_links = full_indivisible_df['google_link'].tolist()
id_list = [link.split("&usg=",1)[1] for link in google_links]


In [50]:
time_regex = re.compile(r'\d{1,2}(?:(?:am|pm)|(?::\d{1,2})(?:am|pm)?)', re.IGNORECASE)

dates = []
start_times = []
end_times = []
cities = []
states = []
legislators = []
legislator_handles = []
phone_numbers = []
titles = []

for i, desc in enumerate(descriptions):
    date = ''
    start_time = ''
    end_time = ''
    city = ''
    state = ''
    legislator = ''
    legislator_handle = ''
    phone_number = ''
    url = ''
    title = ''
    
    doc = nlp(desc)
    all_dates = [doc.text for doc in doc.ents if doc.label_ == 'DATE']
    date_matches = re.findall(date_include_regex, ' '.join(all_dates))
    ### EXCLUDE SOME DIRTY DATES FROM TWITTER THAT SPACY MISTAKENLY INCLUDES    
    if date_matches:
        date = all_dates[0]
    dates.append(date)
        
    times = re.findall(time_regex,desc)
    if times:
        start_time = times[0]
    if len(times) > 1:
        end_time = times[1]
    start_times.append(start_time)
    end_times.append(end_time)
    
    desc_cities = re.findall(city_regex,desc)
    if desc_cities:
        desc_cities = list(set([city.title() for city in desc_cities]))
        if len(desc_cities) == 1:
            city = desc_cities[0]
        else:
            city = '; '.join(desc_cities)
    cities.append(city)
    
    desc_states = re.findall(state_regex,desc)
    if desc_states:
        desc_states = list(set(desc_states))
        if len(desc_states) == 1:
            state = desc_states[0]
        else:
            state = '; '.join(desc_states)
    states.append(state)
    
    desc_legislators = re.findall(leg_name_regex,desc)
    if desc_legislators:
        if len(desc_legislators) == 1:
            legislator = desc_legislators[0]
        else:
            legislator = '; '.join(desc_legislators)
    legislators.append(legislator)
    
    desc_leg_handles = re.findall(leg_twitter_regex,desc)
    if desc_leg_handles:
        if len(desc_leg_handles) == 1:
            legislator_handle = desc_leg_handles[0]
        else:
            legislator_handle = '; '.join(desc_leg_handles)
    legislator_handles.append(legislator_handle)
    
    if phonenumbers.PhoneNumberMatcher(desc, "US"):
        for i,match in enumerate(phonenumbers.PhoneNumberMatcher(desc, "US")):
            if i == 0:  
                phone_number = phonenumbers.format_number(match.number,
                                                            phonenumbers.PhoneNumberFormat.NATIONAL)
    phone_numbers.append(phone_number)
    
    title = names[i]
    titles.append(title)

In [51]:
city_states = []

for i, city in enumerate(cities):
    city_state = ''
    state = states[i]
    if city != '':
        if state == '':
            city_state = city
        elif state != '':
            city_state = '; '.join(list(set([cit + ', ' + state.split('; ')[0] for cit in city.split('; ')])))
        else:
            city_state = city
    else:
        if state == city:
            city_state = city
        elif state != '':
            city_state = state
            
    if city_state == 'Washington, Washington':
        city_state = 'Washington, DC'
    if city_state == 'Mobile':
        city_state = ''
    city_states.append(city_state)

In [52]:
for i, city_state in enumerate(city_states[:1000]):
    print (city_state)
    print (names[i])
    print ()

Long Beach, CA
Indivisible Long Beach CA - Home | Facebook

Long Beach, CA
Indivisible Long Beach CA | Facebook

Huntington Beach, CA; Orange County, CA
Indivisible Orange County (Huntington Beach, CA) | Meetup

Long Beach, CA
Indivisible Neighbors - INDIVISIBLE CA-D39

Phoenix, AZ
Indivisible Phoenix - Home | Facebook

Kansas City, Kansas
Indivisible Kansas City - Home

Kansas City, Kansas
Indivisible KC - MO5 - Home | Facebook

Omaha, Nebraska
Indivisible Omaha - Home | Facebook

Cleveland, OH
Cleveland, OH! - Indivisible Guide | Facebook

Columbus, Ohio
Indivisible Columbus | Resisting the Trump Agenda in Columbus, Ohio

Virginia
Indivisible Virginia - Home | Facebook

Roanoke, VA; Manassas, VA; Lexington, VA; Virginia Beach, VA
Indivisible Virginia

Miami, Florida; Ames, Florida
Indivisible Miami - Home | Facebook

Miami, FL
Indivisible Miami-Dade and Broward - Home | Facebook

Miami, Florida
Indivisible Miami-Dade and Broward Public Group | Facebook

Miami, Florida
Indivisible Sou

In [53]:
print (len(id_list))
print (len(names))
print (len(descriptions))
print (len(urls))
print (len(legislators))
print (len(legislator_handles))
print (len(cities))
print (len(states))
print (len(city_states))
print (len(phone_numbers))

408
408
408
408
408
408
408
408
408
408


In [54]:
#### ORGS SEND TO SQL

orgs_to_sql = pd.DataFrame(
                        {'org_id': id_list,
                         'org_title': [name.encode('utf-8') for name in names],
                         'org_description': [desc.encode('utf-8') for desc in descriptions],
                         'org_url': [url.encode('utf-8') for url in urls],
                         'org_legislators': legislators,
                         'org_legislator_twitter': legislator_handles,
                         'org_city':cities,
                         'org_state':states,
                         'org_city_state':city_states,
                         'org_phone_number': phone_numbers,
                         'org_n_members': '',
                         'org_lat':'',
                         'org_lon':''
                        })


In [55]:
orgs_to_sql = orgs_to_sql[['org_id',
                         'org_title',
                         'org_description',
                         'org_url',
                         'org_legislators',
                         'org_legislator_twitter',
                         'org_city',
                         'org_state',
                         'org_city_state',
                         'org_phone_number',
                         'org_n_members',
                         'org_lat',
                         'org_lon'
                        ]]


orgs_to_sql[:100]

,org_id,org_title,org_description,org_url,org_legislators,org_legislator_twitter,org_city,org_state,org_city_state,org_phone_number,org_n_members,org_lat,org_lon
0,AFQjCNGrewpZhfF4A3QQLxZ6nT9x-q3auw,b'Indivisible Long Beach CA - Home | Facebook',b'Indivisible Long Beach CA - Home | Facebook ...,b'https://www.facebook.com/IndivisibleLongBeac...,,,Long Beach,CA,"Long Beach, CA",,,,
1,AFQjCNHZ74Qel_c9dfcI1k-arKUWf8JpZQ,b'Indivisible Long Beach CA | Facebook',b'Indivisible Long Beach CA | Facebook Indivis...,b'https://www.facebook.com/IndivisibleLongBeac...,,,Long Beach,CA,"Long Beach, CA",,,,
2,AFQjCNHGsRVws-dBt9vP7iqFykbs80qJqg,"b'Indivisible Orange County (Huntington Beach,...","b'Indivisible Orange County (Huntington Beach,...",b'https://www.meetup.com/Indivisible-Orange-Co...,,,Huntington Beach; Orange County,CA,"Huntington Beach, CA; Orange County, CA",,,,
3,AFQjCNG249GTt7NOsjnm5lUIsAQvmt8emw,b'Indivisible Neighbors - INDIVISIBLE CA-D39',b'Indivisible Neighbors - INDIVISIBLE CA-D39 ....,b'https://indivisibleca-d39.org/indivisible-ne...,,,Long Beach,CA,"Long Beach, CA",,,,
4,AFQjCNFugMXJMr1OFzoXc_vBCqD-vS36Tg,b'Indivisible Phoenix - Home | Facebook',b'Indivisible Phoenix - Home | Facebook AZ Act...,b'https://www.facebook.com/indivisiblephoenix/',,,Phoenix,AZ,"Phoenix, AZ",,,,
5,AFQjCNFg_TmvAjb5QrrxwKVYNcwQOGkmFg,b'Indivisible Kansas City - Home',b'Indivisible Kansas City - Home Welcome to In...,b'http://www.indivisiblekc.com/',,,Kansas City,Kansas; MO,"Kansas City, Kansas",(816) 884-3411,,,
6,AFQjCNG8u-pr-7F_LK10CYDQMGhC976n-A,b'Indivisible KC - MO5 - Home | Facebook',b'Indivisible KC - MO5 - Home | Facebook Indiv...,b'https://www.facebook.com/IndivisibleMO5/',,,Kansas City,Kansas; MO; Missouri,"Kansas City, Kansas",,,,
7,AFQjCNE3dB6Bn89W0MBTdHTI_Boc-CcAwg,b'Indivisible Omaha - Home | Facebook',b'Indivisible Omaha - Home | Facebook Indivisi...,b'https://www.facebook.com/standindivisibleoma...,,,Omaha,Nebraska,"Omaha, Nebraska",,,,
8,AFQjCNHC0camPY5wq_SLWjVlsbC3OUh0Jg,"b'Cleveland, OH! - Indivisible Guide | Facebook'","b'Cleveland, OH! - Indivisible Guide | Faceboo...",b'https://www.facebook.com/indivisibleguide/ph...,,,Cleveland,OH,"Cleveland, OH",,,,
9,AFQjCNGLu6HFh062fN5GsUR-MiWA9qNVPw,b'Indivisible Columbus | Resisting the Trump A...,b'Indivisible Columbus | Resisting the Trump A...,b'http://indivisiblecolumbus.com/',,,Columbus,Ohio,"Columbus, Ohio",,,,


In [58]:
orgs_to_sql.to_sql('rzst_groups',conn,if_exists='replace',index=False)
